## Competative Social Media Analysis

**Explore Data Science Academy** wants to develop a marketing startegy to ensure that all social media content posted reaches the inteded target market or audiance and that our social media market are growing in engagement and also analysing our growing social media engagement as compared to the competitors.

#### Import Standard libraries

In [1]:
import re
import csv
import pandas as pd

import selenium
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt

# for extraction
from geograpy import extraction
import nltk
nltk.download('maxent_ne_chunker')

# for organization
from collections import Counter

import time
from datetime import datetime


[nltk_data] Downloading package maxent_ne_chunker to /home/nano-
[nltk_data]     ai/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


#### Configurations

In [2]:
import warnings
warnings.filterwarnings('ignore')

#Disable chrome Notifications
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")

#Install latest chrome drivers
#driver = webdriver.chrome(service=Service(ChromeDriverManager().install()))

#Thanks stackoverflow @pythonjar
#chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

#specify the path to chromedriver.exe (download and save on your computer)
browser = webdriver.Chrome('/home/nano-ai/chromedriver' , chrome_options=chrome_options)

### Automatic Login to Social Media

In [3]:
#Get the Company's  URL from User
page = input("Enter the Company Linkedin URL: ")
company_name = page[33:-1]

#See if existing user credential file exists or create one 
try:
    f= open("linkedin_credentials.txt","r")
    contents = f.read()
    username = contents.replace("=",",").split(",")[1]
    password = contents.replace("=",",").split(",")[3]
except:
    f= open("linkedin_credentials.txt","w+")
    username = input('Enter your linkedin username: ')
    password = input('Enter your linkedin password: ')
    f.write("username={}, password={}".format(username,password))
    f.close()


Enter the Company Linkedin URL: https://www.linkedin.com/school/udacity/posts/?feedView=all


#### Navigate to facebook page/group

In [4]:
#accessing Chromedriver
# browser = webdriver.Chrome('chromedriver')
browser.maximize_window()
time.sleep(2)

#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)
elementID.submit()

In [5]:
#Go to company post webpage
browser.get(page)


#Simulate scrolling to capture all posts
SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

### Check page source code

In [11]:
#Check out page source code
company_page = browser.page_source  


#Use Beautiful Soup to get access tags
linkedin_soup = bs(company_page.encode("utf-8"), "html")
linkedin_soup.prettify()

#Find the post blocks
containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})

### Data Scraping

In [8]:
<div class="inline-block">
      <div class="org-top-card-summary-info-list__info-item">
        Mountain View, California
      </div>

      <div class="org-top-card-summary-info-list__info-item">
        78,255+ alumni
      </div>

        <div class="org-top-card-summary-info-list__info-item">
          339,802 followers
        </div>
  </div>

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [65]:
# Lists that we will iterate to
post_dates = []
post_texts = []
post_likes = []
post_comments = []
post_shares = []
media_links = []
media_type = []
company = []
followers = []

#Looping through the posts and appending them to the lists
for container in containers:
    
    #Try function to make sure its a post and not a promotion
    try:
        posted_date = container.find("span",{"class":"visually-hidden"})
        text_box = container.find("div",{"class":"feed-shared-text relative feed-shared-update-v2__commentary"})
        text = text_box.find("span",{"dir":"ltr"})
        new_likes = container.findAll("span", {"class":"social-details-social-counts__reactions-count"})
        new_comments = container.findAll("li", {"class": "social-details-social-counts__item social-details-social-counts__comments social-details-social-counts__item--with-social-proof"})
        company_name = container.findAll("span" , {"class":"feed-shared-actor__name t-14 t-bold hoverable-link-text t-black"})
        num_followers = container.findAll("span" , {"feed-shared-actor__description t-12 t-normal t-black--light"})

        
        #Appending date and text to lists
        post_dates.append(posted_date.text.strip())
        post_texts.append(text_box.text.strip())


        #Determining media type and collecting relevant info for each type
        try:
            video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
            video_link = video_box[0].find("video", {"class":"vjs-tech"})
            media_links.append(video_link['src'])
            media_type.append("Video")
        except:
            try:
                image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
                media_links.append(image_link['src'])
                media_type.append("Image")
            except:
                try:
                    image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                    image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
                    media_links.append(image_link['src'])
                    media_type.append("Image")
                except:
                    try:
                        article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
                        article_link = article_box[0].find('a', href=True)
                        media_links.append(article_link['href'])
                        media_type.append("Article")
                    except:
                        try:
                            video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
                            video_link = video_box[0].find('a', href=True)
                            media_links.append(video_link['href'])
                            media_type.append("Youtube Video")   
                        except:
                            try:
                                poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                                media_links.append("None")
                                media_type.append("Other: Poll, Shared Post, etc")
                            except:
                                media_links.append("None")
                                media_type.append("Unknown")



        #Getting Video Views. (The folling three lines prevents class name overlap)
        view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
        view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
        result = view_container2 - view_container1

        view_container = []
        for i in result:
            view_container += i

        try:
            post_shares.append(view_container[1].text.strip())

        except:
            post_shares.append(0)

        
        #Appending likes and comments if they exist
        try:
            post_likes.append(new_likes[0].text.strip())
        except:
            post_likes.append(0)
            pass

        try:
            post_comments.append(new_comments[0].text.strip())
        except:                                                           
            post_comments.append(0)
            
            pass
        try:
            company.append(company_name[0].text.strip())
            
        except:
            company.append(0)
            
        try:
            followers.append(num_followers[0].text.strip())

        except:
            followers.append(0)
    
    except:
        pass


In [66]:
#Stripping non-numeric values
comment_count = []
for i in post_comments:
    s = str(i).replace('Comment','').replace('s','').replace(' ','')
    comment_count += [s]

In [67]:
data = {
    "Company": company,
    "Date Posted": post_dates,
    "Media Type": media_type,
    "Post Text": post_texts,
    "Post Likes": post_likes,
    "Post Comments": comment_count,
    "Post Shares": post_shares,
    "Media Links": media_links,
    "Followers":followers
}

df = pd.DataFrame(data)

In [68]:
df.tail(50)

,Company,Date Posted,Media Type,Post Text,Post Likes,Post Comments,Post Shares,Media Links,Followers
17,Udacity,2 weeks ago,"Other: Poll, Shared Post, etc","This is your chance to get $1,000+ worth of pr...",32,1comment,1 comment,None,"339,805 followers"
18,Udacity,1 week ago,Image,Udacity & Niya have partnered to launch a prog...,96,6comment,6 comments,https://media-exp1.licdn.com/dms/image/C5622AQ...,"339,805 followers"
19,Udacity,2 weeks ago,Article,Social media is constantly changing and it’s h...,27,0,3 shares,https://bit.ly/3DjbQ0x,"339,805 followers"
20,Udacity,1 month ago,Article,3 Benefits of Learning #DataAnalytics:1. Data ...,49,1comment,1 comment,https://bit.ly/3JwqMKP,"339,805 followers"
21,Udacity,1 month ago,Image,Excited to announce that I'll be speaking at #...,7,0,0,https://media-exp1.licdn.com/dms/image/C5622AQ...,"339,805 followers"
22,Udacity,3 weeks ago,Image,We're teaming up with the ALXT company to offe...,223,22comment,22 comments,https://media-exp1.licdn.com/dms/image/C5622AQ...,"339,805 followers"
23,Udacity,3 weeks ago,Image,Do you want to build #programming and #webdeve...,50,1comment,1 comment,https://media-exp1.licdn.com/dms/image/C5622AQ...,"339,805 followers"
24,Udacity,2 months ago,Image,That feeling when hard work pays off 💪. Udacit...,130,1comment,16 shares,https://media-exp1.licdn.com/dms/image/C5622AQ...,"339,805 followers"
25,Udacity,3 weeks ago,Image,Do you want to build #programming and #webdeve...,61,6comment,18 shares,https://media-exp1.licdn.com/dms/image/C4D22AQ...,"339,805 followers"
26,Udacity,3 weeks ago,Article,80% of enterprise companies think their L&D pr...,26,0,2 shares,https://bit.ly/3uqDZPj,"339,805 followers"


In [ ]:
data['Company'][0]

### Data Preprocessing

#### Convert Dataframe to CSV File

In [69]:
#Exporting as csv file to program folder
df.to_csv("{}_posts.csv".format(df['Company'][0]), encoding='utf-8', index=False)

#### Save csv file to Coud Storage (Aws RDS)